In [2]:
#!/bin/python

# ----------------- #
# LOAD DEPENDENCIES
# ----------------- #

import sys
import os
import pandas as pd
import numpy as np

if '__file__' in globals():
    grandparent_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(__file__))))
else:
    grandparent_dir = os.getcwd()

sys.path.append(grandparent_dir)
print("Grandparent Directory:", grandparent_dir)
print("sys.path:", sys.path)
from funcs import preprocessing

Grandparent Directory: /Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/preprocessing_scripts
sys.path: ['/Library/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/.venv/lib/python3.13/site-packages', '/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/preprocessing_scripts', '/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/preprocessing_scripts']


ModuleNotFoundError: No module named 'funcs'

In [3]:
dataset = 'MV2014'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/e22-sup-tables1.csv', header=0)
print('Raw data loaded.')

data.columns = data.columns.str.strip()
print(f"Dataset Columns: {data.columns}")
print(data.head())  # Print first few rows to inspect data

# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization prob'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Positions within proteins'].str.contains(';', na=False)]
data = data[~data['Gene names'].str.contains(';', na=False)]


ERROR: Error in parse(text = input): <text>:8:8: unexpected string constant
7: data.columns = data.columns.str.strip()
8: print(f"Dataset Columns: {data.columns}"
          ^


In [ ]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')

preprocessing.match_seq_to_genename(data, 'Sequence window')
print('Amino acid sequences matched to gene names.')

In [ ]:
print("Before creating Phosphosite, columns:", data.columns)

# Ensure GeneName exists before proceeding
if 'GeneName' not in data.columns:
    raise ValueError("GeneName column is missing! Check match_seq_to_genename function.")


data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'

# Debugging: Check if 'Phosphosite' column is present after creation
if 'Phosphosite' not in data.columns:
    print("Error: 'Phosphosite' column not created!")
else:
    print("Phosphosite column created successfully.")

print("After creating Phosphosite, columns:", data.columns)  # Check if 'Phosphosite' is added


In [ ]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Ratio' in col]
data = data[keepcols]

print("Data after subsetting columns:", data)
print("Cols after subsetting:", data.columns)

In [ ]:
# log2 transform the ratio columns (Ratio columns)
Ratio_columns = [col for col in data.columns if 'Ratio' in col]
data[Ratio_columns] = data[Ratio_columns].apply(pd.to_numeric, errors='coerce')
print("After transformation:")
print(data.head())  # Show the first few rows after processing

In [ ]:
data = preprocessing.create_phos_ID(data) # call function to create phosphosite_ID column
print('Phosphosite IDs created.')

In [ ]:
ata = preprocessing.clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
print(data.head())


In [ ]:
final_columns = ['phosphosite_ID'] + [col for col in data.columns if 'Ratio' in col]
data = data[final_columns]
print("Final dataset preview:")
print(data.head())  # Display first few rows
print(data.tail())  # Display last few rows

In [ ]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/MV2014.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)